In [2]:
from gensim.models import Word2Vec
from khaiii import KhaiiiApi; k = KhaiiiApi()
from konlpy.tag import Okt; t = Okt()
from konlpy.utils import pprint
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
raw_data = pd.read_csv('./new_train.csv')
test = pd.read_csv('../../data/dev.hate.csv')
train = raw_data.copy()

X_train = train['comments']
X_test = test['comments']
y_train = train['label']
y_test = test['label']

### 학습을 위한 댓글 데이터(200만개) 로드 후 10만개만 추출

In [4]:
with open('../../../korean-hate-speech-detection/unlabeled_comments.txt', 'r') as f:
    text = f.readlines()
text = [txt.replace("\n", "") for txt in text]
text[:10]

['지드래곤은 난봉꾼이란...댓글도 달렸네 ㅋㅋ 이주연 학창시절 사진 보고 와라. 요즘 웬만한 여자 연예인하고 붙여놔도....미모가 최고였단다.ㅋ 5대 얼짱 출신.',
 '이주연은 알겠는데 지디는 뭐하는 듣보잡여',
 '부럽네요. 나도 불과 한달전까진 허니문베이비를 꿈꿨는데 이제 다 부질없네요. 당연히 순결할거라 믿었고 그래서 첫날밤까지 기다려준건데 배신감만 듭니다. 첫날밤 와이프가 피를 안흘렸어요. 처가집식구들이 일부러 절 속였단 생각에 화도나고 어제 처가집가기로 했는데 안간다고 했더니 혼자 울고 갔다와서 지금까지 한마디도 안해요. 이혼하고 싶네요',
 '"이주연을 모르는 애들이 많네. 해체된 애프터스쿨 멤버로 당시는 주연이 예명. 인기나 포텐은 안터졌으나, 순수미모만으로는 애프터스쿨에서 원탑이었다. 진짜 자연미인이다."',
 '겨론했으면',
 '이주연이 아깝다 진심',
 '방탄은 건드리지말자 인간적으로.. 해외활동 지금주터 시작인데 터지면 진짜 전 세계적으로 활짝 피기도 전에 난리난다.. 국위선양하는 떠오르는 샛별은 제발 건드리지말고.. 탄이들도 만약 연애하더라도 들키지 않길.. ㅠㅠ',
 '선남선녀의 만남이네요^^',
 '문재앙 또뭘덥고 싶어서 ㄷㄷㄷ',
 '주연이 아깝긴해요']

In [5]:
len(text)

2034837

### 전처리

In [8]:
# 특수문자 제거
import re
def cleanse(text):
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    return text

text = [cleanse(txt) for txt in text]

# 띄어쓰기
from pykospacing import spacing
text = [spacing(txt) for txt in text]

# 문장 분리
import kss
# train['comments'] = train['comments'].apply(kss.split_sentences)
# train['comments'] = [','.join(map(str, ls)) for ls in train['comments']]
text = [kss.split_sentences(txt) for txt in text]

# 중복 제거
from soynlp.normalizer import *
# train['comments'] = [repeat_normalize(comment, num_repeats=2) for comment in train['comments']]
text = [repeat_normalize(txt, num_repeats=2) for txt in text]

TypeError: expected string or bytes-like object

In [ ]:
tokenized_data = []
for sentence in text:
    tokenized_data.append(t.morphs(sentence, stem=True)

In [ ]:
embedding = word2vec.Word2Vec(tokenized_data, size=100, window=5, min_count=5, workers=4)
embedding.save('./comments100.model')
# sentences : 각 문장 마다 하나의 토큰 list를 생성하며 토큰 list의 개수는 문장 개수 n개 만큼 생성하여 sentences에 저장해둠
# size : word 벡터 차원
# window : 현재 단어와 예측 단어의 최대 거리
# negative : 0으로 두면 negative smapling을 하지하지 않으며, 0보다 큰 값이면 negative sampling을 수행함
# min_count : min_count의 빈도수보다 낮은 빈도수인 단어는 무시합니다.
# worker : 모델 생성시 사용할 쓰레드 개수
# sg : sg 값이 1이면 skip-gram이지만 그렇지 않으면 CBOW 알고리즘을 이용

In [8]:
model = Word2Vec.load('./million_comments.model')

In [7]:
model.most_similar(positive=["문재인"], topn=30)

[('탄핵', 0.8041871786117554),
 ('적폐', 0.7653246521949768),
 ('국정', 0.7122087478637695),
 ('근해', 0.6986802816390991),
 ('쇼통', 0.6714324951171875),
 ('은박', 0.6534347534179688),
 ('농단', 0.6520810127258301),
 ('명박', 0.6471959352493286),
 ('촛불시위', 0.6338386535644531),
 ('대통령', 0.6292534470558167),
 ('방관', 0.6271576285362244),
 ('적폐임', 0.6238600015640259),
 ('방심위', 0.6228931546211243),
 ('정치', 0.622541606426239),
 ('정부', 0.61871337890625),
 ('석탄', 0.6154335737228394),
 ('화이트리스트', 0.6152961254119873),
 ('지지율', 0.6125301718711853),
 ('확증', 0.6124624013900757),
 ('속필', 0.6098626852035522),
 ('박근혜정부', 0.6084606647491455),
 ('찬조', 0.6065130233764648),
 ('부풀리기', 0.6002240180969238),
 ('언론', 0.5973242521286011),
 ('척결', 0.5949002504348755),
 ('봉착', 0.5943487882614136),
 ('남평문씨', 0.5917057991027832),
 ('친박', 0.5914238691329956),
 ('평화상', 0.5912672281265259),
 ('정으니', 0.5902544856071472)]

In [9]:
set(model.wv.index2word)

{'졸잼',
 '은혜',
 '상한',
 '프라하',
 '백수',
 '각양각색',
 '엇냐',
 '에궁',
 '초딩',
 '헌금',
 'ㅋㅋㅁㅊ',
 '좌빠',
 '본질',
 '어벤저스',
 '3000',
 '보신탕',
 '베니',
 '쉨',
 '괜찬음',
 '매매',
 '답변',
 '필수',
 '경사',
 '갑진',
 '라곤',
 '찌끄레기들',
 '스쿨존',
 'ㄱㄹㅇ',
 '주환',
 '뭐합',
 '베이글',
 '외우다',
 '살충제',
 '묘연하다',
 '괴들',
 '탈색',
 '유니클로',
 '장물아비',
 '이냐',
 '라그',
 '순결하다',
 '냉정하다',
 '쾌차',
 '마지못하다',
 '중절',
 '잼있슴',
 '만들엇네',
 '러버',
 '개연',
 '달자',
 'Joong',
 '카시트',
 '일리',
 '폰질',
 '제기차기',
 '졸렬하다',
 '파도',
 '시멘트',
 '실전',
 '앞길',
 '오바이트',
 '계산기',
 'Are',
 '인스피릿',
 '용감한형제',
 '듀스',
 '방조',
 '차이나다',
 '간척',
 'his',
 '바이킹스',
 '경매',
 '다난',
 '형준',
 '유쾌하',
 '포동',
 '뻔뻔스럽다',
 '저해',
 '야인',
 '생리통',
 '허위',
 '는가',
 '곰탕',
 '즉시',
 'rm',
 '오심',
 '한여름',
 '안식',
 '진로',
 '괜찬다',
 '수지니',
 '매다',
 '데이브레이크',
 '어게인',
 '렷',
 '껍질',
 '모에',
 '꾐',
 '김연지',
 '등하교',
 '디스커버리',
 '예언',
 '논공읍',
 '광보',
 '째째하다',
 '김수현',
 '흉터',
 '옌예인',
 '넥션',
 '남아도',
 '청혼',
 '가트',
 '매하',
 '부은',
 '바나나',
 '볼땐',
 '막달',
 '본바탕',
 '남창희',
 '걸캅스',
 '지구인',
 '인절미',
 '순간포착',
 '오르락내리락',
 '혈팬',
 '그르케',
 '완선',
 '형돈',
 '승낙',
 '바라봄

In [6]:
def get_features(words):
    # 출력 벡터 초기화
    feature_vector = np.zeros(100, dtype=np.float32)
    num_words = 0
    
    # 어휘 사전 준비
    index2word_set = set(model.wv.index2word)
    
    for w in words:
        if w in index2word_set:
            num_words = 1
            # 사전에 해당하는 단어에 대해 단어 벡터를 더함
            feature_vector = np.add(feature_vector, model[w])
            
    # 문장의 단어 수만큼 나누어 단어 벡터의 평균값을 문장 벡터로 함
    feature_vector = np.divide(feature_vector, num_words)
    return feature_vector

In [7]:
get_features('아 진짜 짜증난다')

array([-0.07574642, -0.0988207 , -0.48576212, -0.4776301 ,  1.0557545 ,
        1.638185  , -0.52016985, -0.10827143, -0.6731411 ,  0.5465597 ,
       -1.2510335 ,  0.7781881 ,  0.86021054, -1.5637627 , -1.1958218 ,
        0.48384106, -0.5856546 ,  0.3942331 ,  0.58857167, -0.8910247 ,
       -0.86850727,  1.3836409 ,  0.14626695,  0.72884285,  0.14109324,
       -0.9455564 , -1.2405579 , -0.9072786 ,  0.1578528 , -0.3662238 ,
        0.9585849 ,  0.27965495, -0.5208655 ,  0.61863285,  0.80589837,
       -0.7552631 , -0.7169019 , -0.13069305,  0.49273053,  0.58437   ,
       -0.9490637 ,  0.21465805, -1.1149619 , -2.0163708 , -0.53309685,
        1.2184591 , -0.40251458, -1.573397  ,  0.21312025, -0.6301665 ,
       -0.09679359, -0.5476607 , -1.2148906 ,  0.9498155 ,  1.4136295 ,
       -0.6151596 , -0.06888026, -0.26032174, -0.1605458 ,  1.1390946 ,
        0.05094002, -0.39497793, -0.60097647, -0.42819184, -0.88350314,
       -0.45874068,  0.06497264,  1.0659873 ,  0.55221194,  0.54

In [8]:
def get_dataset(comments):
    dataset = []
    
    for s in comments:
        dataset.append(get_features(s))
        
    commentFeatureVecs = np.stack(dataset)
    return commentFeatureVecs

In [10]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(multi_class='multinomial', class_weight='balanced')

X_train_vecs = get_dataset(X_train)
lr.fit(X_train_vecs, y_train)

X_test_vecs = get_dataset(X_test)

preds = lr.predict(X_test_vecs)
len(preds)

471

In [11]:
from sklearn.metrics import accuracy_score, f1_score
print("Accuracy : {}".format(accuracy_score(preds, y_test)),
    "F1 Score : {}".format(f1_score(preds, y_test, average='macro')))

Accuracy : 0.5520169851380042 F1 Score : 0.5457512516497584


In [12]:
test['preds'] = preds
test

,comments,label,preds
0,송중기 시대극은 믿고본다. 첫회 신선하고 좋았다.,none,none
1,지현우 나쁜놈,offensive,none
2,알바쓰고많이만들면되지 돈욕심없으면골목식당왜나온겨 기댕기게나하고 산에가서팔어라,hate,hate
3,설마 ㅈ 현정 작가 아니지??,hate,offensive
4,이미자씨 송혜교씨 돈이 그리 많으면 탈세말고 그돈으로 평소에 불우이웃에게 기부도 좀...,offensive,none
...,...,...,...
466,지현우 범죄 저지르지 않았나요?,offensive,none
467,여자인생 망칠 일 있나 ㅋㅋ,hate,hate
468,근데 전라도에서 사고가 났는데 굳이 서울까지 와서 병원에 가느 이유는?,offensive,none
469,"할매젖x, 뱃살x, 몸매 s라인, 유륜은 적당해야됨(너무크거나 너무 작아도 x), ...",hate,hate


In [13]:
test[test['label'] != test['preds']]

,comments,label,preds
1,지현우 나쁜놈,offensive,none
3,설마 ㅈ 현정 작가 아니지??,hate,offensive
4,이미자씨 송혜교씨 돈이 그리 많으면 탈세말고 그돈으로 평소에 불우이웃에게 기부도 좀...,offensive,none
8,알았어 그만,offensive,none
11,아c발 어쩌라고 뭔기사가계속나오냐,hate,offensive
...,...,...,...
463,강용석도 찌질하네 과거들추기는 추하다,offensive,none
464,지겹이 이프로 그만하자 작가야,offensive,none
466,지현우 범죄 저지르지 않았나요?,offensive,none
468,근데 전라도에서 사고가 났는데 굳이 서울까지 와서 병원에 가느 이유는?,offensive,none


In [16]:
from sklearn.ensemble                import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model            import LogisticRegression
from sklearn.model_selection         import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics                 import accuracy_score, recall_score, roc_auc_score, f1_score
from sklearn.pipeline                import Pipeline
from sklearn.svm                     import SVC
from lightgbm                        import LGBMClassifier
import warnings
warnings.filterwarnings(action='ignore')

In [17]:
models = [RandomForestClassifier(), LogisticRegression(), SVC(), LGBMClassifier()]

In [18]:
for model in models:
    vec_model = model

    vec_model.fit(X_train_vecs, y_train)
    preds = vec_model.predict(X_test_vecs)

    print(model, "\n",
          "Accuracy : {}".format(accuracy_score(preds, y_test)), '\n'
          "F1 Score : {}".format(f1_score(preds, y_test, average='macro')), '\n', sep="")

RandomForestClassifier()
Accuracy : 0.4437367303609342
F1 Score : 0.40657653491000234

LogisticRegression()
Accuracy : 0.5201698513800425
F1 Score : 0.49559043496640337

SVC()
Accuracy : 0.4819532908704883
F1 Score : 0.451411480568302

LGBMClassifier()
Accuracy : 0.5031847133757962
F1 Score : 0.48263028571247757



In [ ]:
model = Word2Vec.load('./comments100.model')

In [21]:
train = pd.read_csv('./new_train.csv')
dev = pd.read_csv('../../data/dev.hate.csv')
test = pd.read_csv('../../data/test.hate.no_label.csv')
train = pd.concat([train, dev], axis=0)
X_train = train['comments']
X_test = test['comments']
y_train = train['label']
# y_test = test['label']

In [22]:
X_train_vecs = get_dataset(X_train)

lr = LogisticRegression(multi_class='multinomial', class_weight='balanced')
lr.fit(X_train_vecs, y_train)

X_test_vecs = get_dataset(X_test)

preds = lr.predict(X_test_vecs)
len(preds)

974

In [23]:
test['label'] = preds
test.head()

,comments,label
0,ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ,none
1,둘다 넘 좋다~행복하세요,none
2,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,none
3,원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요,none
4,장현승 얘도 참 이젠 짠하다...,none


In [24]:
def kaggle_format(df):
    df['label'][df['label'] == 'none'] = 0
    df['label'][df['label'] == 'offensive'] = 1
    df['label'][df['label'] == 'hate'] = 2
    return df

In [25]:
kaggle_format(test)

,comments,label
0,ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ,0
1,둘다 넘 좋다~행복하세요,0
2,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,0
3,원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요,0
4,장현승 얘도 참 이젠 짠하다...,0
...,...,...
969,대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹,2
970,성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...,2
971,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,2
972,입에 손가릭이 10개 있으니 징그럽다,2


In [26]:
test.to_csv('./0117_jc.csv', index=False)